In [1]:
import numpy as np
from ase import Atoms
from ase.build import molecule
from ase.io import write
from ase.visualize import view

# --- パラメータ設定 ---

# 1. 作成する水分子の数
num_water_molecules = 100

# 2. 目標とする密度 (g/cm^3)
#    水の密度は約 1.0 g/cm^3 です
target_density_g_cm3 = 1.0

# 3. 出力ファイル名
output_file = "water_box.xyz"

# --- スクリプト本体 ---

# 水分子オブジェクトを作成
h2o = molecule("H2O")

# 水のモル質量 (g/mol)
# H: 1.008 * 2, O: 15.999
molar_mass_h2o = 2 * 1.008 + 15.999

# アボガドロ定数 (mol^-1)
avogadro_number = 6.022e23

# Å^3 から cm^3 への変換係数 (1 Å = 1e-8 cm)
angstrom3_to_cm3 = (1e-8) ** 3

# 密度をASEの単位系 (g/Å^3) に変換
# density (g/Å^3) = density (g/cm^3) * (1 cm^3 / 1 Å^3)
target_density_g_angstrom3 = target_density_g_cm3 * angstrom3_to_cm3

# 1分子あたりの質量 (g)
mass_per_molecule_g = molar_mass_h2o / avogadro_number

# 1分子あたりの体積 (Å^3) を計算
# volume = mass / density
volume_per_molecule_angstrom3 = mass_per_molecule_g / target_density_g_angstrom3

print(f"--- 設定情報 ---")
print(f"作成する水分子の数: {num_water_molecules}")
print(f"目標密度: {target_density_g_cm3} g/cm^3")
print(f"1分子あたりの計算体積: {volume_per_molecule_angstrom3:.2f} Å^3")
print("-" * 20)


# 指定した密度と分子数に基づいて、セル内に分子をランダムに配置
# mxn_approx(molecules, n, density)
# molecules: 配置する分子のリスト
# n: 分子の総数
# density: 1分子あたりの体積 (Å^3)
# water_box = mxn_approx([h2o], [num_water_molecules], volume_per_molecule_angstrom3)

# セルに周期境界条件を設定
water_box.pbc = True

# --- 結果の確認と保存 ---

# 作成されたセルの情報を表示
print("\n--- 作成されたセルの情報 ---")
print(f"総原子数: {len(water_box)}")
print("セルベクトル (Å):")
print(water_box.get_cell())
print(f"セルの体積: {water_box.get_volume():.2f} Å^3")

# 実際の密度を計算して確認
actual_mass_g = num_water_molecules * mass_per_molecule_g
actual_volume_cm3 = water_box.get_volume() * angstrom3_to_cm3
actual_density_g_cm3 = actual_mass_g / actual_volume_cm3
print(f"作成されたセルの密度: {actual_density_g_cm3:.4f} g/cm^3")


# 作成した構造をファイルに保存 (xyz形式)
# LAMMPSのdataファイルなど、他の形式にも変更可能です
write(output_file, water_box, format="xyz")
print(f"\n構造を '{output_file}' に保存しました。")

# 作成した構造をビューアで表示（GUI環境がある場合）
# view(water_box)

ImportError: cannot import name 'mxn_approx' from 'ase.build' (/home/jovyan/.py311/lib/python3.11/site-packages/ase/build/__init__.py)

In [20]:
import numpy as np
from ase import Atoms
from ase.build import molecule
from ase.io import read, write
from ase.visualize import view

# --- グローバルなパラメータ設定 ---

# 1. 目標とする密度 (g/cm^3)
target_density_g_cm3 = 1.0

# 2. 単位セルを複製する回数 [x, y, z]
#    例: [5, 5, 5] は 5x5x5 = 125個の単位セルを並べる
replication = (5,5,3)

# 3. 出力ファイル名
output_file_unit_cell = "water_unit_cell.xyz"
output_file_final = "water_box_repeated.xyz"


# ===================================================================
# STEP 1: 水分子1つを含む小さな「単位セル」を作成する
# ===================================================================

print("--- STEP 1: 単位セルの作成 ---")

# --- 定数 ---
molar_mass_h2o = 2 * 1.008 + 15.999  # 水のモル質量 (g/mol)
avogadro_number = 6.022e23         # アボガドロ定数 (mol^-1)
angstrom3_to_cm3 = (1e-8) ** 3     # Å^3 から cm^3 への変換係数

# --- 単位セルの体積計算 ---
mass_per_molecule_g = molar_mass_h2o / avogadro_number
target_density_g_angstrom3 = target_density_g_cm3 * angstrom3_to_cm3
volume_per_molecule_angstrom3 = mass_per_molecule_g / target_density_g_angstrom3

# 立方体のセルを仮定し、体積から一辺の長さを計算
side_length = volume_per_molecule_angstrom3**(1/3.0)
cell_vec = [side_length, side_length, side_length]

print(f"目標密度から計算された単位セルの体積: {volume_per_molecule_angstrom3:.2f} Å^3")
print(f"立方体セルの辺の長さ: {side_length:.2f} Å")

# --- 単位セルの構築 ---
# 1. まず水分子オブジェクトを作成
unit_cell = molecule("H2O")

# 2. 計算した大きさのセルを設定
unit_cell.set_cell(cell_vec)

# 3. 分子がセルの中心に来るように移動
unit_cell.center()

# 4. 周期境界条件を有効にする
unit_cell.pbc = True

# 5. 単位セルをファイルに保存
write(output_file_unit_cell, unit_cell, format="xyz")
print(f"構造を '{output_file_unit_cell}' に保存しました。")
print("-" * 20)


# ===================================================================
# STEP 2: 単位セルを読み込み、指定回数だけ複製する
# ===================================================================

print("\n--- STEP 2: 単位セルの複製 ---")

# 1. 先ほど保存した単位セルを読み込む
base_cell = unit_cell

# 2. .repeat() メソッドでセルを複製
#    元のセルの原子とセル情報が指定された回数だけ複製される
final_box = base_cell.repeat(replication)

print(f"単位セルを {replication} のように複製しました。")

# --- 結果の確認と保存 ---
print("\n--- 作成された最終セルの情報 ---")
print(f"総分子数: {final_box.get_number_of_atoms() // 3}")
print(f"総原子数: {len(final_box)}")
print("セルベクトル (Å):")
print(final_box.get_cell())
print(f"セルの体積: {final_box.get_volume():.2f} Å^3")

# 実際に作成されたセルの密度を計算して確認
num_molecules = np.prod(replication)
actual_mass_g = num_molecules * mass_per_molecule_g
actual_volume_cm3 = final_box.get_volume() * angstrom3_to_cm3
actual_density_g_cm3 = actual_mass_g / actual_volume_cm3
print(f"作成されたセルの密度: {actual_density_g_cm3:.4f} g/cm^3")

# 最終的な構造をファイルに保存
write(output_file_final, final_box, format="xyz")
print(f"\n最終的な構造を '{output_file_final}' に保存しました。")


# 最終的な構造をビューアで表示（GUI環境がある場合）
# try:
#     view(final_box)
# except Exception as e:
#     print(f"\nビューアの表示に失敗しました: {e}")

--- STEP 1: 単位セルの作成 ---
目標密度から計算された単位セルの体積: 29.92 Å^3
立方体セルの辺の長さ: 3.10 Å
構造を 'water_unit_cell.xyz' に保存しました。
--------------------

--- STEP 2: 単位セルの複製 ---
単位セルを (5, 5, 3) のように複製しました。

--- 作成された最終セルの情報 ---
総分子数: 75
総原子数: 225
セルベクトル (Å):
Cell([15.521529313564288, 15.521529313564288, 9.312917588138573])
セルの体積: 2243.65 Å^3
作成されたセルの密度: 1.0000 g/cm^3

最終的な構造を 'water_box_repeated.xyz' に保存しました。


/tmp/ipykernel_40/2459683035.py:80: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  print(f"総分子数: {final_box.get_number_of_atoms() // 3}")


In [1]:
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None

def build_interface(slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        print(f"    1. 界面を構築中...")
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) |(position1[:, 0] < 0) | (position1[:, 1] >= target_xy[1])|(position1[:, 1] < 0) )[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])|(position2[:, 0] < 0) |(position2[:, 1] < 0) )[0]
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        interface = cut_slab1 + cut_slab2
        # interface.cell = [12, 12, 53]
        return interface

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(


In [5]:
 run_matlantis_optimization(atoms=final_box, trajectory_path="water443_opt.traj", fmax=0.05,)

  -> Matlantis最適化開始 (structure, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -484.603 eV
  -> 💾 最適化後の中間構造を保存します: water443_opt.xyz


Atoms(symbols='H96O48', pbc=True, cell=[12.41722345085143, 12.41722345085143, 9.312917588138573], calculator=ASECalculator(...))

In [7]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read,write
pvdf2=read(".xyz") 
pvdf=pvdf2.copy()
positions=pvdf.get_positions()
cell=[positions[:,1].max(),positions[:,2].max(),positions[:,0].max()]
pvdf.positions[:,0]-=positions[:,0].min()
pvdf.positions[:,1] -=positions[:,1].min()
pvdf.positions[:,2] -=positions[:,2].min()
write("pvdf_editopt3.xyz",pvdf)

In [19]:
# pvdf.set_cell(cell)
# pvdf.rotate(10, 'x', rotate_cell=False)
show_gui(pvdf)
write("pvdf_editopt4.xyz",pvdf)
# pvdf.get_positions

In [23]:
# cell
positions

array([[ 1.11697145e+01, -1.28607983e+00,  1.70179292e+01],
       [ 1.20122585e+01, -8.81498200e-02,  1.69971184e+01],
       [ 1.17294935e+01,  6.74251400e-01,  1.58464548e+01],
       [ 1.33585565e+01, -4.63536400e-01,  1.68972936e+01],
       [ 1.19337055e+01,  8.76430880e-01,  1.82260137e+01],
       [ 1.06645495e+01,  1.60908280e+00,  1.83606925e+01],
       [ 1.02299395e+01,  2.84569013e+00,  1.76557931e+01],
       [ 1.12647185e+01,  3.96799307e+00,  1.76255444e+01],
       [ 1.27625915e+01,  1.58609358e+00,  1.81035113e+01],
       [ 1.21248045e+01,  2.60957990e-01,  1.91184704e+01],
       [ 9.60479449e+00,  8.58204720e-01,  1.88006600e+01],
       [ 9.94722649e+00,  2.62202780e+00,  1.66087069e+01],
       [ 9.31933249e+00,  3.23049229e+00,  1.81370872e+01],
       [ 1.23288965e+01,  3.60213008e+00,  1.67968441e+01],
       [ 1.18285895e+01,  4.10801290e+00,  1.88999740e+01],
       [ 1.07826215e+01,  5.36051076e+00,  1.72259133e+01],
       [ 9.97881649e+00,  5.50521624e+00

In [10]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read,write
# pvdf2=read("pvdf_editopt4.xyz") 
big_pvdf=read("pvdf_big222.xyz")
# pvdf=pvdf2.copy()
# positions=pvdf.get_positions()
# cell=[13,18,positions[:,0].max()]
# # pvdf.positions[:,0] =-positions[:,0].min()
# final_box=read("water_box_repeated.xyz")
final_box=read("300H2O_4.xyz")
# pvdf.set_cell(cell)
# # positions[:,1].max(),positions[:,2].max(),positions[:,0].max(),
# big_pvdf=pvdf.repeat((4,4,1))
# # show_gui(big_pvdf)
big_final_box=final_box.copy()
big_final_box.set_cell((big_final_box.positions[:,0].max(),big_final_box.positions[:,1].max(),big_final_box.positions[:,2].max()))
# big_final_box=big_final_box.repeat((2,2,1))
interface=build_interface(big_final_box, big_pvdf, target_xy=(26.3, 36.7), separation=0.5)
write("pvdf_water_random2.xyz",interface)
# positions=pvdf.get_positions()
# positions[:,1].max(),positions[:,2].max(),positions[:,0].max(),

    1. 界面を構築中...


In [25]:
cell

[20.539054840000002, 17.40730359, 13.35855649]

In [6]:
pvdf_water=read("pvdf_water_random.xyz")
# position1=pvdf_water.get_positions()
cut_slab1=pvdf_water.copy()
# indices_to_delete1 = np.where((position1[:, 2] >= 20))[0]
# if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
cut_slab1.pbc=False
run_matlantis_optimization(atoms=cut_slab1, trajectory_path="pvdf_water_random_opt.traj", fmax=0.05)
traj=Trajectory("pvdf_water_random_opt.traj")[-1]
write("pvdf_water_random_opt.xyz",traj)

  -> Matlantis最適化開始 (structure, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -22616.729 eV
  -> 💾 最適化後の中間構造を保存します: pvdf_water_random_opt.xyz


In [60]:
pvdf2=read("pvdf_editopt4.xyz") 
pvdf=pvdf2.copy()
positions=pvdf.get_positions()
cell=[13,18,positions[:,0].max()]
pvdf.set_cell(cell)
big=pvdf.repeat((2,2,2))
big.positions[:,0].max(),big.positions[:,1].max(),big.positions[:,2].max()
# write(filename="pvdf_big222.xyz",images=big) 

(26.358556489999998, 36.27643515, 32.97088761)

In [55]:
write(filename="pvdf_big20.xyz",images=big) 

In [ ]:
## -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
# INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 100000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 600        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用
# --- 緩和計算の設定 ---
RELAX_TEMP = 300.0      # 緩和計算の温度 (K)
N_RELAX_STEPS = 5000    # 緩和計算のステップ数
RELAX_TRAJ_FILE_NAME = "relax_pvdf_water_random2_600K.traj"
# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "pvdf_water_random2_600K.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    
    # 3. MDシステムの初期化
    # atoms=Trajectory("pvdf_water_random_opt.traj")[-1]
    atoms=read("pvdf_water_random2.xyz")
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")
    integrator_relax=NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=300,
            taut=100.0 * units.fs  # 温度緩和時間
        )
    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return
    # 5. MDFeatureの設定
    md_relax = MDFeature(
        integrator=integrator_relax,
        n_run=1000,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=RELAX_TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    # md_results = md_relax(system)
    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

システムを 600 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (100000 ステップ) ---

--- MDシミュレーションを開始します (100000 ステップ) ---


  0%|          | 0/100000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/pvdf_water_random2_600K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-3.12 eV/atom  total energy: -3.05 eV/atom  temperature: 598.74 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:    50  energy：-4.01 eV/atom  total energy: -3.88 eV/atom  temperature: 969.95 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:   100  energy：-4.06 eV/atom  total energy: -3.98 eV/atom  temperature: 648.98 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:   150  energy：-4.07 eV/atom  total energy: -3.99 eV/atom  temperature: 607.55 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:   200  energy：-4.08 eV/atom  total energy: -4.00 eV/atom  temperature: 616.33 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:   250  energy：-4.08 eV/atom  total energy: -4.00 eV/atom  temperature: 599.72 K  volume: 37363 Ang^3  density: 1.170 g/cm^3
steps:   300  energy：-4.08 eV/atom  total energy: -4.01 e

In [ ]:
# -*- coding: utf-8 -*-
import logging
import numpy as np
from ase import units
from ase.build import bulk
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator,AndersenIntegrator,NPTBerendsenIntegrator,NVTBerendsenIntegrator,VelocityVerletIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# Matlantis / PFP
from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- ユーザー設定項目 (ここを編集してください) ---

# 1. 使用するIntegratorの種類を選択
#    選択肢: "NVE", "Langevin", "NVT_Berendsen", "NPT_Berendsen", "NPT"VelocityVerletIntegrator
# LangevinIntegrator
# AndersenIntegrator
# NVTBerendsenIntegrator
# NPTBerendsenIntegrator
# NPTIntegrator
INTEGRATOR_TYPE = "NVT_Berendsen"

# 2. 初期構造のファイルパスを指定 (ASEが読み込める形式ならOK)
#    例: "POSCAR", "input.cif", "structure.xyz"
#    指定しない場合は、Siのバルク結晶で計算します。
# INPUT_STRUCTURE_FILE ="pvdf_opt2.traj"  # "your_structure_file.cif" など

# 3. MDシミュレーションの基本パラメータ
N_RUN_STEPS = 100000          # 実行するステップ数
TIME_STEP_FS = 1.0         # タイムステップ (fs)
TEMPERATURE_K = 400        # 温度 (K)
PRESSURE_PA = 101325       # 圧力 (Pa) ※NPT系でのみ使用
# --- 緩和計算の設定 ---
RELAX_TEMP = 300.0      # 緩和計算の温度 (K)
N_RELAX_STEPS = 5000    # 緩和計算のステップ数
# RELAX_TRAJ_FILE_NAME = "relax_pvdf_big222_water_700K.traj"
# 4. ログとトラジェクトリの設定
SHOW_PROGRESS = True       # 進捗バーを表示するか
SHOW_LOGGER = True         # ログを標準出力に表示するか
LOGGER_INTERVAL = 50       # ログを出力する間隔 (ステップ数)
TRAJ_FILE_NAME = "pvdf_big222_water_400K.traj"  # 出力トラジェクトリファイル名
TRAJ_FREQ = 50             # トラジェクトリを保存する間隔 (ステップ数)
vacuum=0.2
# 5. PFPモデルの計算設定
PFP_MODEL_VERSION = 'v7.0.0'
PFP_CALC_MODE = EstimatorCalcMode.CRYSTAL_U0

# --- 設定項目ここまで ---


def select_integrator(integrator_type: str):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す関数。
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")
    
    # NPT系で使用する圧力 (ASE unitsに変換)
    pressure_au = PRESSURE_PA * units.Pascal

    if integrator_type == "NVE":
        # Microcanonical (NVE) ensemble using the Velocity Verlet algorithm
        return VelocityVerletIntegrator(timestep=TIME_STEP_FS)
        
    elif integrator_type == "Langevin":
        # Langevin dynamics for NVT ensemble
        return LangevinIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            friction=0.002  # 摩擦係数 (調整が必要な場合あり)
        )

    elif integrator_type == "NVT_Berendsen":
        # NVT ensemble with Berendsen thermostat
        return NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            taut=100.0 * units.fs  # 温度緩和時間
        )
        
    elif integrator_type == "NPT_Berendsen":
        # NPT ensemble with Berendsen thermostat and barostat
        return NPTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            taut=100.0 * units.fs,    # 温度緩和時間
            taup=1000.0 * units.fs    # 圧力緩和時間
        )
        
    elif integrator_type == "NPT":
        # NPT ensemble with Nosé-Hoover thermostat and Parrinello-Rahman barostat
        return NPTIntegrator(
            timestep=TIME_STEP_FS,
            temperature=TEMPERATURE_K,
            pressure=pressure_au,
            ttime=25.0 * units.fs,    # 温度緩和時間 (Nosé-Hoover)
            pfactor=75.0 * units.fs  # 圧力緩和時間 (Parrinello-Rahman)
        )
        
    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation():
    """
    設定に基づいてMDシミュレーションを実行するメイン関数。
    """
    # Matlantis-featuresのロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # 1. PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=PFP_MODEL_VERSION, calc_mode=PFP_CALC_MODE)
    
    # 3. MDシステムの初期化
    atoms=read("pvdf222_water.xyz")
    positions = atoms.get_positions()
    min_coords = positions.min(axis=0)
    max_coords = positions.max(axis=0)
    
    # 4. 新しいセルの大きさを計算
    #    (分子の最大幅) + (両側の真空層)
    cell_lengths = (max_coords - min_coords) + 2 * vacuum
    new_cell = np.diag(cell_lengths)
    
    # 5. 計算したセルをatomsオブジェクトに設定
    atoms.set_cell(new_cell)
    atoms.pbc=True
    system = ASEMDSystem(atoms)
    
    # 乱数シードを固定して温度を初期化（再現性のため）
    system.init_temperature(temperature=TEMPERATURE_K, stationary=True, rng=np.random.RandomState(seed=12345))
    print(f"システムを {TEMPERATURE_K} K に初期化しました。")
    integrator_relax=NVTBerendsenIntegrator(
            timestep=TIME_STEP_FS,
            temperature=300,
            taut=100.0 * units.fs  # 温度緩和時間
        )
    # 4. Integratorの選択と準備
    try:
        integrator = select_integrator(INTEGRATOR_TYPE)
    except ValueError as e:
        print(e)
        return
    # 5. MDFeatureの設定
    md_relax = MDFeature(
        integrator=integrator_relax,
        n_run=1000,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=RELAX_TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    # md_results = md_relax(system)
    # 5. MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=N_RUN_STEPS,
        show_progress_bar=SHOW_PROGRESS,
        show_logger=SHOW_LOGGER,
        logger_interval=LOGGER_INTERVAL,
        traj_file_name=TRAJ_FILE_NAME,
        traj_freq=TRAJ_FREQ,
        estimator_fn=estimator_fn,
    )

    # 6. MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({N_RUN_STEPS} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{TRAJ_FILE_NAME}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")


if __name__ == "__main__":
    run_md_simulation()

システムを 400 K に初期化しました。
--- Integrator: 'NVT_Berendsen' を選択しました ---

--- MDシミュレーションを開始します (100000 ステップ) ---

--- MDシミュレーションを開始します (100000 ステップ) ---


  0%|          | 0/100000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/pvdf_big222_water_400K.traj.
Note: The max disk size of /home/jovyan is about 98G.
steps:     0  energy：-3.43 eV/atom  total energy: -3.37 eV/atom  temperature: 400.88 K  volume: 40085 Ang^3  density: 1.154 g/cm^3
steps:    50  energy：-4.04 eV/atom  total energy: -3.96 eV/atom  temperature: 642.80 K  volume: 40085 Ang^3  density: 1.154 g/cm^3


In [27]:
show_gui(interface)

In [ ]:
build_interface(slab1, slab2, target_xy=(12.0, 12.0), separation=2.0)